## Object Representations
- `repr()`: Return a string representing the object as the developer wants to see it.
- `str()`: Return a string representing the object as the user wants to see it.
- `bytes()`: Return a byte sequence representing the ibject


The special methods `__repr__`, `__str__` and `__bytes__()` support `repr()`, `str()` and `bytes()`

## Vector Class

In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    def __iter__(self):
        return (i for i in (self.x, self.y))
    def __repr__(self):
        class_name = type(self).__name__
        return f'{class_name}({self.x!r}, {self.y!r})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [2]:
v1 = Vector2d(3, 4)
x, y = v1
b = bytes(v1)
print(b)
x, y, abs(v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


(3.0, 4.0, 5.0)

In [3]:
Vector2d.frombytes(b)

Vector2d(3.0, 4.0)

### `classmethod` vs. `staticmethod`
- `classmethod` changes the way the method is called, so it receives the class itself as the first argument, instead of an instance. Its most common use is for alternative constructors
- `staticmethod` decorator changes a method so that it receives no special first argument (neither `self` nor `cls`). In essence, a static method is just like a plain function that happens to live in a class body

In [4]:
class Demo:
    @classmethod
    def clsmeth(*args):
        return args # return all positional arguments
    @staticmethod
    def stameth(*args):
        return args

Demo.clsmeth('spam'), Demo.stameth('spam')

((__main__.Demo, 'spam'), ('spam',))

### Formatted Displays
The f-strings, the `format()` built-in function, and the `str.format()` method calls actually `.__format__` method

In [5]:
rate = 7.7
print(f'1 EUR = {rate:.2f} RMB')
print(format(rate, '.2f'))
print('1 EUR = {rate: .2f} RMB'.format(rate=rate))

1 EUR = 7.70 RMB
7.70
1 EUR =  7.70 RMB


In [6]:
from datetime import datetime
now = datetime.now()
print(f"It's now {now:%H:%M:%S}")
print(format(now, '%H:%M:%S'))
print("It's now {:%I:%M %p}".format(now))

It's now 20:56:06
20:56:06
It's now 08:56 PM


### Add `.__format__` method to `Vector2d`
if the format specifier ends with a 'p', we’ll display the vector in polar coordinates: `<r, θ>`

In [7]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    def __iter__(self):
        return (i for i in (self.x, self.y))
    def __repr__(self):
        class_name = type(self).__name__
        return f'{class_name}({self.x!r}, {self.y!r})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
    ######################  NEW  ######
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            # call __abs__
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        # call `__iter__` if `coords = self`
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    ######################  NEW  ######

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    ######################  NEW  ######
    def angle(self):
        return math.atan2(self.y, self.x)
    ######################  NEW  ######

v1 = Vector2d(1, 1)
print(f'{v1:.5fp}')
print(format(v1, '.3ep'))

<1.41421, 0.78540>
<1.414e+00, 7.854e-01>


## A Hashable `Vector2d`
As defined, so far our `Vector2d` instances are unhashable, so we can’t put them in a `set`

To make a Vector2d hashable, we must implement `__hash__` (`__eq__` is also required, and we already have it). We also need to make vector instances **immutable**

In [8]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    ######################  NEW  ######
    def __init__(self, x, y):
        # Use exactly two leading underscores to make an attribute private
        self.__x = float(x)
        self.__y = float(y)
    
    @property # The @property decorator marks the getter method of a property.
    def x(self):
        return self.__x
    @property
    def y(self):
        return self.__y
    ######################  NEW  ######

    def __iter__(self):
        return (i for i in (self.x, self.y))
    def __repr__(self):
        class_name = type(self).__name__
        return f'{class_name}({self.x!r}, {self.y!r})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            # call __abs__
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        # call `__iter__` if `coords = self`
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    ######################  NEW  ######
    def __hash__(self):
        # ^ is the XOR operator
        return hash(self.x) ^ hash(self.y)
    ######################  NEW  ######
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)

v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
print(hash(v1), hash(v2))
set([v1, v2])

7 384307168202284039


{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

You may also implement the `__int__` and `__float__` methods, invoked by the `int()` and `float()` constructors

There’s also a `__complex__` method to support the `complex()` built-in constructor.

## Private and “Protected” Attributes in Python
If you name an instance attribute in the form `__mood` (**two leading** underscores and **zero or at most one trailing** underscore), Python stores the name in the instance `__dict__` prefixed with a leading underscore and the class name, so in the `Dog` class, `__mood` becomes `_Dog__mood`

A single underscore prefix often refers to a protected attribute, meaning that you should not access such attributes from outside the class.

In [9]:
v1.__dict__, v1._Vector2d__x

({'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}, 3.0)

## Saving Memory with `__slots__`
By default, Python stores instance attributes in a per-instance `dict` named `__dict__`. If you are dealing with millions of instances with few attributes, the `__slots__` class attribute can save a lot of memory, by letting the interpreter store the instance attributes in a `tuple` instead of a `dict`.

A `__slots__` attribute defined in a class is not inherited by subclasses.

By defining `__slots__` in the class, you are telling the interpreter: “These are all the instance attributes in this class.”

In [10]:
from array import array
import math

class Vector2d:
    ######################  NEW  ######
    __slots__ = ('__x', '__y')
    ######################  NEW  ######
    typecode = 'd'
    def __init__(self, x, y):
        # Use exactly two leading underscores to make an attribute private
        self.__x = float(x)
        self.__y = float(y)
    
    @property # The @property decorator marks the getter method of a property.
    def x(self):
        return self.__x
    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))
    def __repr__(self):
        class_name = type(self).__name__
        return f'{class_name}({self.x!r}, {self.y!r})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            # call __abs__
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        # call `__iter__` if `coords = self`
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    def __hash__(self):
        # ^ is the XOR operator
        return hash(self.x) ^ hash(self.y)
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)